# Quantum Circuit Simulator

In [3]:
import numpy as np

### Define unitary matrices

In [4]:
isqrt2 = 1/np.sqrt(2) # inverse of the square root of 2

x = np.array([
    [0, 1],
    [1, 0]
])

h = np.array([
    [isqrt2, isqrt2],
    [isqrt2, -isqrt2],
])

cx = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 1],
    [0, 0, 1, 0],
])

sw = np.array([
    [1, 0, 0, 0],
    [0, 0, 1, 0],
    [0, 1, 0, 0],
    [0, 0, 0, 1],
])

### Quantum Circuit Simulator class with methods required for simulation

In [5]:
class QCSimulator:
    def __init__(self, qubits_num, qcirc):
        """
        Takes a quantum circuit program, qcirc. Creates initial state vector
        of qubits_num qubits.
        """
        
        # create initial state vector of qubits_num qubits
        init_state = self.get_ground_state(qubits_num)
        
        # number of gate operations to compute
        operations_num = len(qcirc)
        
        # circuit handler, read circuit, calculate matrix operator
        # multiply the state with the operator and return final state.
        self.circuit_handler(qubits_num, operations_num, init_state, qcirc)
        
        
        
        print(init_state)
        print(operations_num)
        print(qcirc[1]["target"])
        
    
    def get_ground_state(self, qubits_num):
        """Returns a vector of size 2**qubits_num with all zeroes except
        the first element which is 1."""
        ground_state_base = np.array([1, 0])
        ground_state_qnum = ground_state_base
        for i in range(qubits_num - 1):
            ground_state_qnum = np.kron(ground_state_base, ground_state_qnum)
            
        return ground_state_qnum
    
    
    def get_operator(self, qubits_num, gate_unitary, qubits_target=1):
        """Returns a unitary operator of size 2**n x 2**n for a given
        quantum gate and target number of qubits.
        Resizing the gate's martix to the dimension of the state vector"""
        # define 2x2 identity matrix
        I = np.identity(2)
        gate_unitary_post = gate_unitary
        for i in range(qubits_num - 1):
            gate_unitary_post = np.kron(I, gate_unitary_post)
        
        return gate_unitary_post
    
    
    def circuit_handler(self, qubits_num, operations_num, init_state, qcirc):
        """Handles quantum circuit input, managing operations with gates, operators
        and state vectors."""
        # calculate matrix operators
        # gate_unitary = self.get_operator(qubits_num, gate_unitary, target_qubits)
        pass
        
    
    def measure(self):
        """Chooses element from state vector using weighted random and
        returns it's index."""
        pass
    
    
    def get_counts(self):
        """Execute measurements in a loop shots_num times and returns object
        with statistics in the form:
        {
            element_index: number_of_occurrences,
            ...
        }
        only from elements which occurred - returned from measrements."""
        pass
    
    
# qcs = QCSimulator
# a = qcs.get_ground_state(1)
# b = qcs.get_operator(X, 2)
# print(a)
# print(b)

N = 2 # number of qubits for the state vector; 2-qubits state vector

# quantum circuit/program
circ = [
    {"gate": "h", "target": [0]},
    {"gate": "cx", "target": [0, 1]},
]

QCSimulator(N, circ)

[1 0 0 0]
2
[0, 1]
